Funciones
Empezamos por crear funciones para crear y abrir los archivo donde se almacenara la informacion.

In [ ]:
#funcion que crea un archivo con un parametro str que le da nombre y que deja el archivo abierto en modo Append
def abreArchivo(nombre):
    try: # revisa que exista el archivo y sino lo crea
        fichero = (open(str(nombre)+'.txt', 'x')) # intento crear el archivo
        print('se ha CREADO el archivo') # mensaje de exito
        fichero.close()
        print('Se ha CERRADO el archivo')
    except:
        print('Archivo ya existe') # mensaje de error
    try: # lo abre en modo append
        fichero = (open(str(nombre)+'.txt', 'a')) # intento abrir en modo append
        print('Archivo abierto en modo APPEND') # mensaje de exito
    except Exception as e:
        print('Se ha producido un Error al intentar ABRIR el archivo en modo APPEND\n', e)# mensaje de error
    return fichero

# pasa la variable entera para que la cierre y de mensaje de error.
def cierraArchivo(nombre):
    try:
        nombre.close()
        print('Se ha CERRADO el archivo')
    except:
        print('Error')

# Funcion que saca los atributos de cada item
def saca_atributos(items):
    x = len(items)-1
    lista = []
    while x >= 0:
        lista.append(items[x]['attributes'])
        x = x - 1
    return lista

# Funcion que saca los datos geometricos y los inserta junto con el ObjectID de es
def saca_geografia(items):
    x = len(items)-1
    diccionario = {}
    while x >= 0:
        a = items[x]['attributes']
        b = items[x]['geometry']
        diccionario['objectID'] = a['OBJECTID'] 
        diccionario['rings'] = b['rings']
        x = x - 1
    return diccionario

#PRUEBA baseDatos = abreArchivo('baseDatos')
#PRUEBA cierraArchivo(baseDatos)

In [ ]:
import requests

# variables para controlar el while (controlan numero de veces que se repite el bucle)
i = 0
limite = 100
# variables para controlar el URL (controlan que pido en cada request)
offset = 1
count = 1

# variables que forman la URL separadas para realizar pruebas
inicio = 'http://mapas.igme.es/gis/rest/services/Cartografia_Geologica/IGME_MAGNA_50/MapServer/10/query?'
f ='f=json' #Formato en el que pasara los datos
where = '&where=(1%3D1)%20AND%20(1%3D1)' #condicion para la consulta
geometry = '&returnGeometry=true'#si pasa la geometria o no
spatial = '&spatialRel=esriSpatialRelIntersects' #AVERIGUAR
outfields = '&outFields=*' #AVERIGUAR
orderby = '&orderByFields=OBJECTID%20ASC' #como viene ordenada la consulta
outSR = '&outSR=102100' #AVERIGUAR parte del codigo de coordenadas?
resultOffset = '&resultOffset=' #cuantas columnas desde el principio antes de empezar a imprimir
resultRecord = '&resultRecordCount=' #cuantas columnas imprimir

#si no existen creo los archivos y los abro en modo append
BD_objetos = abreArchivo('BD_objetos')
BD_atributos = abreArchivo('BD_atributos')
BD_objGeograf = abreArchivo('BD_objGeograf')

while i < limite:
    url = inicio+f+where+geometry+spatial+outfields+orderby+outSR+resultOffset+str(offset)+resultRecord+str(count)
    res = requests.get(url)
    data = res.json()
    items = data['features']
    
    BD_objetos.write(str(items)) 

    atributos = saca_atributos(items)
    BD_atributos.write(str(atributos))
                     
    geografia = saca_geografia(items)
    BD_objGeograf.write(str(geografia))
                     
    print(offset) # imprime un contador de control para log
    offset += 1 # aumento el offset para pasar al proximo item
    i += 1 # aumento el controller del bucle

#cierro los archivos
cierraArchivo(BD_objetos)
cierraArchivo(BD_atributos)
cierraArchivo(BD_objGeograf)